Binomial Method Bermudan Option pricing

In [2]:
import numpy as np

In [23]:
def binomial_tree_stock(S, T, sigma, n):
    dt = T / n
    u = np.exp(sigma * np.sqrt(dt))
    d = 1 / u
    
    stock_tree = np.zeros((n + 1, n + 1))
    stock_tree[0, 0] = S
    
    for i in range(1, n + 1):
        stock_tree[i, i] = stock_tree[i - 1, i - 1] * u
        for j in range(i, n + 1):
            stock_tree[j, i - 1] = stock_tree[j - 1, i - 1] * d
            
    return stock_tree


def call_pf(S, K):
    return max(S - K, 0)


def put_pf(S, K):
    return max(K - S, 0)


def payoff(S, K, style):
    if style == 'call':
        return call_pf(S, K)
    elif style == 'put':
        return put_pf(S, K)
    else:
        raise ValueError("Invalid option style. Style must be 'call' or 'put'.")

def binomial_bermudan_option_pricing(S, K, T, r, sigma, n, early_exercise_dates, style):
    dt = T / n
    u = np.exp(sigma * np.sqrt(dt))
    d = 1 / u
    p = (np.exp(r * dt) - d) / (u - d)

    # Initialize option value matrix
    option_values = np.zeros((n + 1, n + 1))

    # Calculate the option values at expiration
    for i in range(n + 1):
        option_values[n][i] = payoff(S * (u ** i) * (d ** (n - i)), K, style)

    # Calculate the option values at earlier exercise dates
    for t in range(n - 1, -1, -1):
        for i in range(t + 1):
            if t * dt in early_exercise_dates:
                option_values[t][i] = max(payoff(S * (u ** i) * (d ** (n - i)), K, style), np.exp(-r * dt) * (p * option_values[t + 1][i] + (1 - p) * option_values[t + 1][i + 1]))
            else:
                option_values[t][i] = np.exp(-r * dt) * (p * option_values[t + 1][i] + (1 - p) * option_values[t + 1][i + 1])

    return option_values[0][0]

In [42]:
S0 = 100   # Initial stock price
K = 100   # Strike price
T = 1.0   # Time to maturity
r = 0.09  # Risk-free interest rate
sigma = 0.2  # Volatility
n = 100   # Number of time steps in the binomial tree
early_exercise_dates = [0.5, 0.8]  # List of early exercise dates

bermudan_option_price = binomial_bermudan_option_pricing(S0, K, T, r, sigma, n, early_exercise_dates, 'put')
print("Bermudan Call Option Price:", bermudan_option_price)

Bermudan Call Option Price: 61.29978315118754
